In [3]:
import spacy


In [2]:
import getpass
import os

from google.colab import userdata
sec_key=userdata.get("GEMINI_API")


In [4]:
pip install -qU langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.9 MB/s eta 0:00:00


In [8]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass("your API key")

your API key·


In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,
    max_output_tokens=256,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [10]:
import json

def cv_to_json(cv_text):
    # This function assumes a specific format for the CV text
    # You will need to adjust the parsing logic based on the actual format of your CVs

    cv_lines = cv_text.split('\n')
    cv_data = {}

    for line in cv_lines:
        if line.startswith("Name:"):
            cv_data["name"] = line.split(":")[1].strip()
        elif line.startswith("Contact:"):
            cv_data["contact"] = line.split(":")[1].strip()
        elif line.startswith("Education:"):
            cv_data["education"] = line.split(":")[1].strip()
        elif line.startswith("Experience:"):
            cv_data["experience"] = line.split(":")[1].strip()
        elif line.startswith("Skills:"):
            cv_data["skills"] = [skill.strip() for skill in line.split(":")[1].split(",")]

    return json.dumps(cv_data, indent=4)

# Example usage:
mentee_cv = """
Name: John Doe
Contact: john.doe@example.com
Education: B.Sc. in Computer Science
Experience: 2 years as a software developer
Skills: Python, Java, SQL
"""

mentor_cv = """
Name: Jane Smith
Contact: jane.smith@example.com
Education: M.Sc. in Software Engineering
Experience: 5 years as a senior software engineer
Skills: Python, JavaScript, AWS, Docker
"""

mentee_cv_json = cv_to_json(mentee_cv)
mentor_cv_json = cv_to_json(mentor_cv)

print("Mentee CV JSON:")
print(mentee_cv_json)

print("\nMentor CV JSON:")
print(mentor_cv_json)


Mentee CV JSON:
{
    "name": "John Doe",
    "contact": "john.doe@example.com",
    "education": "B.Sc. in Computer Science",
    "experience": "2 years as a software developer",
    "skills": [
        "Python",
        "Java",
        "SQL"
    ]
}

Mentor CV JSON:
{
    "name": "Jane Smith",
    "contact": "jane.smith@example.com",
    "education": "M.Sc. in Software Engineering",
    "experience": "5 years as a senior software engineer",
    "skills": [
        "Python",
        "JavaScript",
        "AWS",
        "Docker"
    ]
}


In [11]:
mentee_cv = """
Name: John Doe
Contact: john.doe@example.com
Education: B.Sc. in Computer Science
Experience: 2 years as a software developer
Skills: Python, Java, SQL
"""

mentor_cv = """
Name: Jane Smith
Contact: jane.smith@example.com
Education: M.Sc. in Software Engineering
Experience: 5 years as a senior software engineer
Skills: Python, JavaScript, AWS, Docker
"""

messages = [
    (
        "system",
        "You are a helpful assistant that mentors developers. You are to check the similarity between the CV of a mentee and a mentor's CV.",
    ),
    (
        "human",
        f"Here are the CVs of the mentee and the mentor:\n\nMentee's CV: {mentee_cv}\n\nMentor's CV: {mentor_cv}\n\nPlease compare the two CVs and provide a similarity analysis."
    ),
]

ai_msg = llm.invoke(messages)
print(ai_msg)


content="## CV Similarity Analysis: John Doe (Mentee) & Jane Smith (Mentor)\n\nHere's a comparison of John and Jane's CVs, highlighting similarities and differences to guide the mentoring relationship:\n\n**Similarities:**\n\n* **Core Skills:** Both John and Jane share Python as a core programming skill. This provides a strong foundation for mentorship in this area. \n* **Software Development Focus:** Both CVs clearly demonstrate experience and interest in software development, ensuring aligned career goals.\n\n**Differences:**\n\n* **Experience Level:** Jane possesses significantly more experience (5 years) than John (2 years). This gap allows Jane to provide guidance on career progression and navigating the industry.\n* **Specific Skills:** While sharing Python, their skillsets diverge. Jane's expertise in JavaScript, AWS, and Docker offers valuable learning opportunities for John, especially if he wants to explore cloud-based development.\n* **Education:** Jane holds a Master's degr

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

def cv_to_json(cv_text):
    # Simple parsing logic, may need adjustments based on CV format
    cv_lines = cv_text.split('\n')
    cv_data = {}
    for line in cv_lines:
        if line.startswith("Name:"):
            cv_data["name"] = line.split(":")[1].strip()
        elif line.startswith("Contact:"):
            cv_data["contact"] = line.split(":")[1].strip()
        elif line.startswith("Education:"):
            cv_data["education"] = line.split(":")[1].strip()
        elif line.startswith("Experience:"):
            cv_data["experience"] = line.split(":")[1].strip()
        elif line.startswith("Skills:"):
            cv_data["skills"] = [skill.strip() for skill in line.split(":")[1].split(",")]
    return json.dumps(cv_data, indent=4)

mentee_cv = """
Name: John Doe
Contact: john.doe@example.com
Education: B.Sc. in Computer Science
Experience: 2 years as a software developer
Skills: Python, Java, SQL
"""

mentor_cv = """
Name: Jane Smith
Contact: jane.smith@example.com
Education: M.Sc. in Software Engineering
Experience: 5 years as a senior software engineer
Skills: Python, JavaScript, AWS, Docker
"""

# Convert CVs to JSON (optional, if needed for other purposes)
mentee_cv_json = cv_to_json(mentee_cv)
mentor_cv_json = cv_to_json(mentor_cv)

# Convert CVs to a list for vectorization
cvs = [mentee_cv, mentor_cv]

# Vectorize the CVs using TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cvs)

# Calculate cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

# Extract the similarity score between the mentee and mentor CVs
similarity_score = similarity_matrix[0][1]

print(f"Similarity Score: {similarity_score:.2f}")

# Generate the message for the AI
messages = [
    (
        "system",
        "You are a helpful assistant that mentors developers. You are to check the similarity between the CV of a mentee and a mentor's CV.",
    ),
    (
        "human",
        f"Here are the CVs of the mentee and the mentor:\n\nMentee's CV: {mentee_cv}\n\nMentor's CV: {mentor_cv}\n\nPlease compare the two CVs and provide a similarity analysis.\n\nThe calculated similarity score is: {similarity_score:.2f}"
    ),
]

ai_msg = llm.invoke(messages)
print(ai_msg)


Similarity Score: 0.34
content="Here's a breakdown of the similarities and differences between John's and Jane's CVs, along with an interpretation of the similarity score:\n\n**Similarities:**\n\n* **Core Skills:** Both John and Jane share Python as a common programming language skill. This suggests a foundational overlap in their technical abilities.\n* **Industry:** Both CVs point to experience in software development, indicating they work within the same industry.\n\n**Differences:**\n\n* **Experience Level:**  Jane has significantly more experience (5 years) compared to John's 2 years. This difference is typical in a mentor-mentee relationship.\n* **Specific Skills:** While they share Python, their other technical skills differ. Jane's CV highlights expertise in JavaScript, AWS, and Docker, which are more advanced and cloud-focused, while John's lists Java and SQL, suggesting a potentially stronger foundation in backend technologies.\n* **Education:** Jane holds a Master's degree, 

In [13]:
print(ai_msg.content)



Here's a breakdown of the similarities and differences between John's and Jane's CVs, along with an interpretation of the similarity score:

**Similarities:**

* **Core Skills:** Both John and Jane share Python as a common programming language skill. This suggests a foundational overlap in their technical abilities.
* **Industry:** Both CVs point to experience in software development, indicating they work within the same industry.

**Differences:**

* **Experience Level:**  Jane has significantly more experience (5 years) compared to John's 2 years. This difference is typical in a mentor-mentee relationship.
* **Specific Skills:** While they share Python, their other technical skills differ. Jane's CV highlights expertise in JavaScript, AWS, and Docker, which are more advanced and cloud-focused, while John's lists Java and SQL, suggesting a potentially stronger foundation in backend technologies.
* **Education:** Jane holds a Master's degree, while John has a Bachelor's. This differenc